In [1]:
import os
import sys
import pandas as pd
import numpy as np
import json
import time
from datetime import datetime, timedelta
import decimal
from decimal import Decimal
context = decimal.getcontext()
context.rounding = decimal.ROUND_HALF_UP
# #####################
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'dashboard.settings')
django.setup()
# ######################
from apps.configs.models import AbstractProduct # 抽象品項
from apps.watchlists.models import MonitorProfile, WatchlistItem # 因應措施,監控品項
from apps.dailytrans.models import DailyTran # 日交易

In [2]:
stime = time.time()

# 0. 設定今天日期

In [3]:
# 以今天決定監控清單編號
today = '2020-03-29'  # 從request.GET,POST或urls的pattern來的字串
year = int(today[0:4])
month = int(today[-5:-3])
day = int(today[-2:])
today = datetime(year, month, day)
# day =
h = month < 7 and 1 or 2
rocY_h = f'{year-1911}_h{h}'
#
WL_maps = {
    '106_h2': 1,
    '107_h1': 2,
    '107_h2': 3,
    '108_h1': 4,
    '108_h2': 5,
    '109_h1': 6,
    '109_h2': 7,
}
# 監控清單編號
WL_id = WL_maps[rocY_h]
WL_id

6

# 1.監控清單

## 1-1: 抓因應措施 mp-2020h1.yaml
SELECT * FROM public.watchlists_monitorprofile

In [4]:
where = {
    'watchlist_id': WL_id,
    # 'row__isnull': False,
    # 'product_id': 40040, # 絲瓜
}
# ORM取出mp-2020h1的因應措施表
qs = MonitorProfile.objects.filter(**where).values()
df_WL_mp = pd.DataFrame(qs).drop(columns=['id', 'update_time'])
# 增加監控品項及列號
extra_product = [(15, 10), (19, 9), (30002, 99), (60051, 100), (60066, 103)] # 增加長糯, 稻穀, 全部花卉 L, 火鶴花 FB, 文心蘭 FO3
extra_product += [(50063, 73), (59019, 72)] # 香蕉台北一二批發,青香蕉下品()內銷)
extra_product += [(60068, 107)] # 香水百合
# 整理補齊欄位
df_tmp = pd.DataFrame(extra_product, columns=['product_id', 'row'])
df_tmp = df_tmp.reindex(columns=df_WL_mp.columns)
df_WL_mp = df_WL_mp.append(df_tmp).sort_values('product_id')
#
df_WL_mp[:3]

,product_id,watchlist_id,type_id,price,comparator,color,info,action,period,is_active,always_display,row
0,15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0
1,19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0
2,30002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,99.0


## 1-2: 抓監控品項 2020h1.yaml
SELECT * FROM public.watchlists_watchlistitem

SELECT * FROM public.watchlists_watchlistitem_sources

In [5]:
where = {
    'parent_id': WL_id,
    # 'product_id': 40164,
}
# 取出對應監控清單的監控品項id及其來源市場
qs = WatchlistItem.objects.filter(**where).values('product_id', 'sources')  # 主動指定sources才會join
df_WL_items = pd.DataFrame(qs).sort_values('product_id')
#
# 增加香水百合, 香蕉台北一二批發,青香蕉下品()內銷) 的來源市場
extra_product = [
    (60068, 30001), (60068, 30002), (60068, 30003), (60068, 30004), (60068, 30005),
    (50063, 20001), (50063, 20002), # 香蕉台北一二批發
]
s1=10030
s2=20000
id_59019 = 59019 #青香蕉下品(內銷)，注意59004"青香蕉(內銷)"是上品用
tmp = []
for s in range(s1,s2+1):
    tmp.append((id_59019,s))
extra_product += tmp   
# 整理欄位
df_tmp = pd.DataFrame(extra_product, columns=['product_id', 'sources'])
df_WL_items = df_WL_items.append(df_tmp).sort_values('product_id').reset_index(drop=True)
#
df_WL_items

,product_id,sources
0,15,NaN
1,19,NaN
2,10006,10001.0
3,10006,10002.0
4,10007,10001.0
...,...,...
10187,120950,NaN
10188,120954,NaN
10189,120987,NaN
10190,121044,NaN


## 1-3: 抽象品項
SELECT * FROM public.configs_abstractproduct

### (1)抽象總表_全品項

In [6]:
# 先撈抽象總表
qs = AbstractProduct.objects.all().values('id', 'name', 'parent_id')
df_abs = pd.DataFrame(qs)
# 沒有parent就用自己id
where = df_abs['parent_id'].isnull()
df_abs['parent_id'] = np.where(where, df_abs['id'], df_abs['parent_id'])
#
df_abs

,id,name,parent_id
0,1,白米,1.0
1,2,硬種(蓬萊),1.0
2,3,硬種(蓬萊),2.0
3,4,硬種(蓬萊),2.0
4,5,硬秈(在來),1.0
...,...,...,...
2088,130001,肥育乳公牛550公斤以上,130001.0
2089,130002,乳公架仔牛150-200公斤,130002.0
2090,130003,肥育肉用母牛,130003.0
2091,130004,閹公牛,130004.0


### (2)抽象總表_四層父子

In [7]:
# join三次，共四層id,name，八個欄位
df_abs_4 = df_abs.merge(df_abs, left_on='parent_id', right_on='id', how='left', suffixes=('', '_p'))
df_abs_4 = df_abs_4.drop(columns=['id_p']).rename(columns={'name_p': 'parent_name', 'parent_id_p': 'p_parent_id'})
#
df_abs_4 = df_abs_4.merge(df_abs, left_on='p_parent_id', right_on='id', how='left', suffixes=('', '_p'))
df_abs_4 = df_abs_4.drop(columns=['id_p']).rename(columns={'name_p': 'p_parent_name', 'parent_id_p': 'p_p_parent_id'})
#
df_abs_4 = df_abs_4.merge(df_abs, left_on='p_p_parent_id', right_on='id', how='left', suffixes=('', '_p'))
df_abs_4 = df_abs_4.drop(columns=['id_p', 'parent_id_p']).rename(columns={'name_p': 'p_p_parent_name'})
# 
df_abs_4

,id,name,parent_id,parent_name,p_parent_id,p_parent_name,p_p_parent_id,p_p_parent_name
0,1,白米,1.0,白米,1.0,白米,1.0,白米
1,2,硬種(蓬萊),1.0,白米,1.0,白米,1.0,白米
2,3,硬種(蓬萊),2.0,硬種(蓬萊),1.0,白米,1.0,白米
3,4,硬種(蓬萊),2.0,硬種(蓬萊),1.0,白米,1.0,白米
4,5,硬秈(在來),1.0,白米,1.0,白米,1.0,白米
...,...,...,...,...,...,...,...,...
2088,130001,肥育乳公牛550公斤以上,130001.0,肥育乳公牛550公斤以上,130001.0,肥育乳公牛550公斤以上,130001.0,肥育乳公牛550公斤以上
2089,130002,乳公架仔牛150-200公斤,130002.0,乳公架仔牛150-200公斤,130002.0,乳公架仔牛150-200公斤,130002.0,乳公架仔牛150-200公斤
2090,130003,肥育肉用母牛,130003.0,肥育肉用母牛,130003.0,肥育肉用母牛,130003.0,肥育肉用母牛
2091,130004,閹公牛,130004.0,閹公牛,130004.0,閹公牛,130004.0,閹公牛


### (3)兩個yaml檔所有的大小類品項id

In [8]:
# 兩個yaml檔所有的大小類品項id
pids = pd.concat([df_WL_mp['product_id'], df_WL_items['product_id']]).drop_duplicates()
# 加上parent_id
for _ in range(3):
    where = df_abs['id'].isin(pids)
    pids = pd.concat([df_abs[where]['parent_id'], pids]).drop_duplicates().dropna()
#
where = df_abs_4['id'].isin(pids)
df_abs_yaml = df_abs_4[where].sort_values('id').reset_index(drop=True)
#
df_abs_yaml

,id,name,parent_id,parent_name,p_parent_id,p_parent_name,p_p_parent_id,p_p_parent_name
0,1,白米,1.0,白米,1.0,白米,1.0,白米
1,14,長糯,1.0,白米,1.0,白米,1.0,白米
2,15,長糯,14.0,長糯,1.0,白米,1.0,白米
3,17,稻穀,17.0,稻穀,17.0,稻穀,17.0,稻穀
4,18,硬種(蓬萊),17.0,稻穀,17.0,稻穀,17.0,稻穀
...,...,...,...,...,...,...,...,...
158,120987,300g-375g,120985.0,午仔魚,120040.0,午仔魚(養殖),120001.0,養殖類
159,121041,龍虎斑,120001.0,養殖類,120001.0,養殖類,120001.0,養殖類
160,121043,龍虎斑A,121041.0,龍虎斑,120001.0,養殖類,120001.0,養殖類
161,121044,0.6-0.9公斤/尾,121043.0,龍虎斑A,121041.0,龍虎斑,120001.0,養殖類


# 2. 日交易表
SELECT * FROM public.dailytrans_dailytran

## 2-1: 日期處理

In [9]:
# 抓包含 today 前21天，today是最後一天
start = today - timedelta(days=20)  # '2020-02-13'
end = today - timedelta(days=0)  # '2020-03-04'
dates = pd.date_range(start=start, end=end, freq='1D')
#
dates

DatetimeIndex(['2020-03-09', '2020-03-10', '2020-03-11', '2020-03-12',
               '2020-03-13', '2020-03-14', '2020-03-15', '2020-03-16',
               '2020-03-17', '2020-03-18', '2020-03-19', '2020-03-20',
               '2020-03-21', '2020-03-22', '2020-03-23', '2020-03-24',
               '2020-03-25', '2020-03-26', '2020-03-27', '2020-03-28',
               '2020-03-29'],
              dtype='datetime64[ns]', freq='D')

In [10]:
# 21天用qcut均分為三周
dates_W = pd.qcut(dates, 3, labels=["W1", "W2", "W3"])
dates_W = pd.DataFrame({'date': dates, 'W': dates_W})
dates_W = dates_W.astype(str)
#
dates_W

,date,W
0,2020-03-09,W1
1,2020-03-10,W1
2,2020-03-11,W1
3,2020-03-12,W1
4,2020-03-13,W1
5,2020-03-14,W1
6,2020-03-15,W1
7,2020-03-16,W2
8,2020-03-17,W2
9,2020-03-18,W2


In [11]:
# 七天標題
W1_date = dates_W.T.iloc[0,0:7]
W2_date = dates_W.T.iloc[0,7:14]
W3_date = dates_W.T.iloc[0,14:21]
title = list(zip(W1_date,W2_date,W3_date))
title = ['\n'.join(t) for t in title]
title

['2020-03-09\n2020-03-16\n2020-03-23',
 '2020-03-10\n2020-03-17\n2020-03-24',
 '2020-03-11\n2020-03-18\n2020-03-25',
 '2020-03-12\n2020-03-19\n2020-03-26',
 '2020-03-13\n2020-03-20\n2020-03-27',
 '2020-03-14\n2020-03-21\n2020-03-28',
 '2020-03-15\n2020-03-22\n2020-03-29']

## 2-2: 日交易篩選21天及累加過parent的pids

In [12]:
where = {
    'date__gte': start,  # 開始
    'date__lte': end,  # 結束
    'product_id__in': pids, #yaml檔所有id及其祖先
    # 'product_id__in': df_abs['id'], #全品項id
    # 'product_id': 40163  # 限定監控品項id
}
# (1)ORM取出yaml的監控品項之交易行情紀錄
cols_tar = ['date', 'product_id', 'source_id', 'avg_price', 'avg_weight', 'volume']
qs = DailyTran.objects.filter(**where).values(*cols_tar)
df_trans = pd.DataFrame(qs)
# (2)從抽象 df_abs_yaml 撈品項name及父類id
df_trans = df_trans.merge(df_abs_yaml, left_on='product_id', right_on='id', how='left').drop(columns=['id'])
# (3)從監控品項df篩選來源市場
left_on = ['product_id', 'source_id']
right_on = ['product_id', 'sources']
df_trans = df_trans.merge(df_WL_items, left_on=left_on, right_on=right_on, how='inner').drop(columns=['sources'])
# (4)date轉str, 重排
df_trans['date'] = df_trans['date'].astype(str)
df_trans = df_trans.sort_values(by=cols_tar).reset_index(drop=True)
# 
df_trans

,date,product_id,source_id,avg_price,avg_weight,volume,name,parent_id,parent_name,p_parent_id,p_parent_name,p_p_parent_id,p_p_parent_name
0,2020-03-09,15,NaN,43.140000,NaN,NaN,長糯,14.0,長糯,1.0,白米,1.0,白米
1,2020-03-09,19,NaN,21.930000,NaN,NaN,硬種(蓬萊),18.0,硬種(蓬萊),17.0,稻穀,17.0,稻穀
2,2020-03-09,30002,30001.0,57.247431,NaN,95327.0,全部花卉,30001.0,全部花卉,30001.0,全部花卉,30001.0,全部花卉
3,2020-03-09,30002,30002.0,45.473177,NaN,23428.0,全部花卉,30001.0,全部花卉,30001.0,全部花卉,30001.0,全部花卉
4,2020-03-09,30002,30003.0,52.328403,NaN,46999.0,全部花卉,30001.0,全部花卉,30001.0,全部花卉,30001.0,全部花卉
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2662,2020-03-29,120947,NaN,41.100000,NaN,NaN,600-800g,120946.0,吳郭魚105,120007.0,吳郭魚,120001.0,養殖類
2663,2020-03-29,120950,NaN,111.100000,NaN,NaN,600g-800g,120948.0,金目鱸,120021.0,金目鱸,120001.0,養殖類
2664,2020-03-29,120954,NaN,82.800000,NaN,NaN,0.6公斤/尾,120953.0,虱目魚晚撈,120198.0,虱目魚,120001.0,養殖類
2665,2020-03-29,120987,NaN,216.600000,NaN,NaN,300g-375g,120985.0,午仔魚,120040.0,午仔魚(養殖),120001.0,養殖類


In [13]:
# where = df_trans['product_id'].isin([50063])
# where &= df_trans['date'] == '2018-04-22'
# df_trans[where]

In [14]:
# where = df_trans['product_id'].isin([59004])
# where &= df_trans['date'] >= '2018-04-08'
# where &= df_trans['date'] <= '2018-04-14'
# df_trans[where]

## 2-3: 計算日平均及周平均

In [15]:
def g_handle(g_df, by_field):
    by_id = g_df[by_field].iloc[0]
    # 上品只留59004
    if by_id == 50061:
        where = g_df['product_id'] == 59004
        g_df = g_df[where]
    # (1)加總處理===========================================================
    # 每日營業額 total_amount = 每日平均價 avg_price * 每日交易量 total_quantity
    has_volume = g_df['volume'].count() / g_df['volume'].shape[0] > 0.8
    has_weight = g_df['avg_weight'].count() / g_df['avg_weight'].shape[0] > 0.8
    if has_volume and has_weight:
        g_df['total_quantity'] = g_df['avg_weight']*g_df['volume']
        g_df['total_amount'] = g_df['avg_price']*g_df['total_quantity']
    elif has_volume:
        g_df['total_quantity'] = g_df['volume']
        g_df['total_amount'] = g_df['avg_price']*g_df['total_quantity']
    else:
        g_df['total_quantity'] = g_df['avg_price']/g_df['avg_price']
        g_df['total_amount'] = g_df['avg_price']
    # 每日每大類，不分子品項/來源市場，的總交易量及總營業額
    cols_tar = ['date', by_field, 'total_amount', 'total_quantity']
    g_df = g_df[cols_tar].groupby(by=['date', by_field], as_index=False).sum()
    # 【每日平均價】________________________________________________________________________________________
    g_df['D_avg_price_noround'] = g_df['total_amount']/g_df['total_quantity']
    g_df['D_avg_price'] = g_df['D_avg_price_noround'].map(lambda x: float(round(Decimal(x), 1)))  # .round(1)
    # date欄位補滿21天
    g_df = g_df.set_index('date').reindex(dates_W.date).reset_index().rename(columns={'index': 'date'})
    # 三周分組W1,2,3
    g_df = g_df.merge(dates_W, on='date')
    g_df[by_field] = g_df[by_field].fillna(method='ffill').fillna(method='bfill').astype(int)  # 填充id
    # 【每周平均價】________________________________________________________________________________________
    if has_volume or has_weight:
        g_df_W = g_df.groupby(by=['W'])[['total_amount', 'total_quantity']].transform('sum')
        g_df_W.rename(columns={'total_amount': 'W_total_amount', 'total_quantity': 'W_total_quantity'}, inplace=True)
        g_df_W['W_avg_price_noround'] = g_df_W['W_total_amount']/g_df_W['W_total_quantity']
        g_df_W = g_df_W[['W_avg_price_noround']]
    else:
        # 周平均直接日平均的平均
        g_df_W = g_df.groupby(by=['W'])[['D_avg_price_noround']].transform('mean')
        g_df_W.rename(columns={'D_avg_price_noround': 'W_avg_price_noround'}, inplace=True)
    # 21天的日平均及每七天的周平均的21列df
    g_df = pd.concat([g_df, g_df_W], axis=1)
    g_df = g_df[['date', by_field, 'D_avg_price', 'W_avg_price_noround', 'W']].sort_values('date')
    # (2)轉置處理===========================================================
    # 紀錄三周平均
    W_avg_price_noround = g_df['W_avg_price_noround'].iloc[[0, 7, 14]].reset_index(drop=True)
    # 日平均轉成三列
    g_df_W1 = g_df[g_df['W'] == 'W1'][['D_avg_price']].reset_index(drop=True).T
    g_df_W2 = g_df[g_df['W'] == 'W2'][['D_avg_price']].reset_index(drop=True).T
    g_df_W3 = g_df[g_df['W'] == 'W3'][['D_avg_price']].reset_index(drop=True).T
    # 三列垂直併接
    g_df_W123 = pd.concat([g_df_W1, g_df_W2, g_df_W3]).reset_index(drop=True)
    g_df_W123[by_field] = by_id
    g_df_W123['W_avg_price'] = W_avg_price_noround.map(lambda x: float(round(Decimal(x), 1)))
    g_df_W123['W'] = ['W1', 'W2', 'W3']
    cols_tar = [by_field, 'W', 'W_avg_price'] + list(range(7))
    g_df_W123 = g_df_W123[cols_tar]
    # 帶出品項名稱
    g_df_W123 = g_df_W123.merge(df_abs[['id', 'name']], left_on=by_field, right_on='id', how='left').drop(columns=['id'])
    g_df_W123['group_level'] = by_field
    g_df_W123.rename(columns={by_field: 'id'}, inplace=True)
    # 與上週比較
    g_df_W123['W_avg_price_lastweek'] = W_avg_price_noround.shift()
    g_df_W123['delta_lastW'] = (W_avg_price_noround-g_df_W123['W_avg_price_lastweek'])*100/g_df_W123['W_avg_price_lastweek']
    g_df_W123['delta_lastW'] = g_df_W123['delta_lastW'].map(lambda x: float(round(Decimal(x), 1)))
    g_df_W123.drop(columns=['W_avg_price_lastweek'], inplace=True)
    cols_tar = ['id', 'name', 'W', 'W_avg_price', 'delta_lastW'] + list(g_df_W123.columns[3:10]) + ['group_level']
    g_df_W123 = g_df_W123[cols_tar]
    #
    return g_df_W123


# ===================================================================================
# 四個級別都做一次匯總
by_fields = ['product_id', 'parent_id', 'p_parent_id', 'p_p_parent_id']
df_trans_W = []
for level in range(4):
    by_field = by_fields[level]
    tmp = df_trans.groupby(by_field).apply(g_handle, by_field=by_field).reset_index(drop=True)
    df_trans_W.append(tmp)
# ===================================================================================
# 帶有三周周平均及21天日平均之df
df_trans_W = pd.concat(df_trans_W).sort_values(by=['id', 'group_level'], ascending=[True, False]).rename(columns=dict(enumerate(title)))
df_trans_W.drop_duplicates(subset=df_trans_W.columns[:-1], inplace=True)  # 留 group_level最小的
df_trans_W

,id,name,W,W_avg_price,delta_lastW,2020-03-09\n2020-03-16\n2020-03-23,2020-03-10\n2020-03-17\n2020-03-24,2020-03-11\n2020-03-18\n2020-03-25,2020-03-12\n2020-03-19\n2020-03-26,2020-03-13\n2020-03-20\n2020-03-27,2020-03-14\n2020-03-21\n2020-03-28,2020-03-15\n2020-03-22\n2020-03-29,group_level
0,1.0,白米,W1,43.1,NaN,43.1,43.1,43.1,43.1,43.1,43.1,43.1,p_parent_id
1,1.0,白米,W2,43.2,0.1,43.1,43.2,43.2,43.2,43.2,43.2,43.2,p_parent_id
2,1.0,白米,W3,43.2,0.1,43.2,43.2,43.2,43.2,43.2,43.2,43.2,p_parent_id
0,14.0,長糯,W1,43.1,NaN,43.1,43.1,43.1,43.1,43.1,43.1,43.1,parent_id
1,14.0,長糯,W2,43.2,0.1,43.1,43.2,43.2,43.2,43.2,43.2,43.2,parent_id
...,...,...,...,...,...,...,...,...,...,...,...,...,...
184,121044.0,0.6-0.9公斤/尾,W2,211.2,0.0,211.2,211.2,211.2,211.2,211.2,211.2,211.2,product_id
185,121044.0,0.6-0.9公斤/尾,W3,211.2,0.0,211.2,211.2,211.2,211.2,211.2,211.2,211.2,product_id
186,130001.0,肥育乳公牛550公斤以上,W1,112.0,NaN,112.0,NaN,NaN,NaN,NaN,NaN,NaN,product_id
187,130001.0,肥育乳公牛550公斤以上,W2,112.0,0.0,112.0,NaN,NaN,NaN,NaN,NaN,NaN,product_id


## 2-4: 顯示特定品項的三周平均數據

In [16]:
where = True
where &= df_trans_W['W'].isin(['W1','W2','W3'])
# 
which =0
where &= df_trans_W['id'] == [40163,40422,50227,40163, 14, 40422,40387,15,130001,40002,90004,120198,120311,3001, 60068,50063,59019,50061,59004][which]
tmp = df_trans_W[where]
tmp

,id,name,W,W_avg_price,delta_lastW,2020-03-09\n2020-03-16\n2020-03-23,2020-03-10\n2020-03-17\n2020-03-24,2020-03-11\n2020-03-18\n2020-03-25,2020-03-12\n2020-03-19\n2020-03-26,2020-03-13\n2020-03-20\n2020-03-27,2020-03-14\n2020-03-21\n2020-03-28,2020-03-15\n2020-03-22\n2020-03-29,group_level
48,40163.0,甘藍,W1,8.1,NaN,NaN,7.8,8.3,8.0,7.8,7.7,9.0,parent_id
49,40163.0,甘藍,W2,9.2,14.0,NaN,9.5,8.8,8.6,8.9,9.9,9.4,parent_id
50,40163.0,甘藍,W3,8.4,-8.2,NaN,9.5,9.2,8.4,7.9,7.6,7.9,parent_id


## 2-6: 價量日報表

In [17]:
# 只留因應措施中有row的品項
tmp = df_WL_mp[['product_id', 'row']][df_WL_mp['row'].notna()].astype({'row':int})
df_DailyReport = df_trans_W.merge(tmp, left_on='id', right_on='product_id', how='right')
df_DailyReport = df_DailyReport.drop(columns=['id', 'group_level'])
cols_tar = ['row', 'name', 'product_id', 'W', 'W_avg_price', 'delta_lastW'] + list(df_DailyReport.columns[4:11])
# # 只留第三周
where = df_DailyReport['W'].isin([np.nan, 'W3'])
df_DailyReport = df_DailyReport[where][cols_tar].drop(columns=['W']).sort_values('row').reset_index(drop=True)
df_DailyReport.columns = list(df_DailyReport.columns[:5]) + W3_date.tolist()
# # 存excel
fn = f'{today.date()}_日報表_pandas.xlsx'
df_DailyReport.replace(np.nan,'-').astype(str).to_excel(fn) 
print(f'已存{fn}')
# 
df_DailyReport

已存2020-03-29_日報表_pandas.xlsx


,row,name,product_id,W_avg_price,delta_lastW,2020-03-23,2020-03-24,2020-03-25,2020-03-26,2020-03-27,2020-03-28,2020-03-29
0,9,硬種(蓬萊),19,22.2,1.1,22.1,22.1,22.2,22.2,22.2,22.2,22.2
1,10,長糯,15,43.2,0.1,43.2,43.2,43.2,43.2,43.2,43.2,43.2
2,14,NaN,40160,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,15,NaN,49002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,16,大蒜,40438,87.9,-1.8,88.5,87.8,87.6,87.6,87.6,87.6,88.3
5,17,洋蔥,40422,12.1,-4.2,NaN,12.9,12.6,11.7,11.7,11.9,11.7
6,19,甘藍,40163,8.4,-8.2,NaN,9.5,9.2,8.4,7.9,7.6,7.9
7,20,包心白,40185,9.2,-5.7,NaN,10.6,9.5,8.9,8.5,8.8,8.7
8,21,青蔥-日蔥,40539,59.3,-24.0,NaN,74.5,66.3,53.6,56.1,53.1,53.8
9,23,青蔥-粉蔥（含北蔥）,40541,25.8,-20.9,NaN,31.2,28.6,25.5,24.1,22.3,22.8


In [18]:
time.time()-stime

13.588887453079224

# 3. 單一品項測試 g_handle

In [ ]:
# 大類甘藍40163，三小類LA1,LA2,LA7
by_field = 'parent_id' 
_id = [40163, 14, 40422,40387,50061][-1]
# 
where = True
where &= df_trans[by_field] == _id
if _id == 50061:
    where &= df_trans['product_id']==59004
g_df = df_trans[where]
#
# (1)加總處理===========================================================
has_volume = g_df['volume'].count() / g_df['volume'].shape[0] > 0.8
has_weight = g_df['avg_weight'].count() / g_df['avg_weight'].shape[0] > 0.8
# 每日營業額 total_amount = 每日平均價 avg_price * 每日交易量 total_quantity
if has_volume and has_weight:
    g_df['total_quantity'] = g_df['avg_weight']*g_df['volume']
    g_df['total_amount'] = g_df['avg_price']*g_df['total_quantity']
elif has_volume:
    g_df['total_quantity'] = g_df['volume']
    g_df['total_amount'] = g_df['avg_price']*g_df['total_quantity']
else:
    g_df['total_quantity'] = g_df['avg_price']/g_df['avg_price']
    g_df['total_amount'] = g_df['avg_price']
# 每日每大類，不分子品項/來源市場，的總交易量及總營業額
cols_tar = ['date', by_field, 'total_amount', 'total_quantity']
g_df = g_df[cols_tar].groupby(by=['date', by_field], as_index=False).sum()
# 【每日平均價】____________________
g_df['D_avg_price'] = (g_df['total_amount']/g_df['total_quantity']).round(1)
# date欄位補滿21天
g_df = g_df.set_index('date').reindex(dates_W.date).reset_index().rename(columns={'index': 'date'})
# 三周分組W1,2,3
g_df = g_df.merge(dates_W, on='date')
g_df[by_field] = g_df[by_field].fillna(method='ffill').fillna(method='bfill').astype(int)
# # 【每周平均價】____________________
g_df_W = g_df.groupby(by=['W'])[['total_amount', 'total_quantity']].transform('sum')
g_df_W.rename(columns={'total_amount': 'W_total_amount', 'total_quantity': 'W_total_quantity'}, inplace=True)
g_df = pd.concat([g_df, g_df_W], axis=1)
g_df['W_avg_price'] = (g_df['W_total_amount'].round(2)/g_df['W_total_quantity']).round(1)
# # #
# g_df = g_df[['date', by_field, 'D_avg_price', 'W_avg_price', 'W']].sort_values('date')
# g_df_W = g_df.groupby(by=['W'])[['D_avg_price']].transform('mean')
# g_df_W.rename(columns={'D_avg_price': 'W_avg_price'}, inplace=True)
# g_df = pd.concat([g_df, g_df_W], axis=1)
# g_df = g_df[['date', by_field, 'D_avg_price', 'W_avg_price', 'W']].sort_values('date')
# 
g_df

In [ ]:
# (2)轉置處理===========================================================
by_id = g_df[by_field][0]
W_avg_price = g_df[['W_avg_price']].iloc[[0, 7, 14], ].values  # array([15. , 14.5,  9.3])
#
g_df_W1 = g_df[g_df['W'] == 'W1'][['D_avg_price']].reset_index(drop=True).T
g_df_W2 = g_df[g_df['W'] == 'W2'][['D_avg_price']].reset_index(drop=True).T
g_df_W3 = g_df[g_df['W'] == 'W3'][['D_avg_price']].reset_index(drop=True).T
#
g_df_W123 = pd.concat([g_df_W1, g_df_W2, g_df_W3]).reset_index(drop=True)
g_df_W123[by_field] = by_id
g_df_W123['W_avg_price'] = W_avg_price
g_df_W123['W'] = ['W1', 'W2', 'W3']
cols_tar = [by_field, 'W', 'W_avg_price'] + list(range(7))
g_df_W123 = g_df_W123[cols_tar]
# #
g_df_W123 = g_df_W123.merge(df_abs[['id', 'name']], left_on=by_field, right_on='id', how='left').drop(columns=['id'])
g_df_W123['group_level'] = by_field
g_df_W123.rename(columns={by_field: 'id'}, inplace=True)
# 
g_df_W123